In [ ]:
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns
import qgrid
%matplotlib inline
sns.set_style('whitegrid')

In [ ]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf

# For Notebooks
init_notebook_mode(connected=True)

# For offline use
cf.go_offline()

In [ ]:
df = pd.read_csv('zen_2018-10-25_44B39350.csv')
for col in df.columns:
    if col != col.strip():
        df[col.strip()] = df[col]
        df.drop([col], inplace=True, axis=1)


In [ ]:
to_float = lambda i: float(i.replace(',', '.'))
df['income'] = df['income'].apply(to_float)
df.drop(df[df['income'] > 0].index, inplace=True)
df.drop([
    'outcomeCurrencyShortTitle',  
    'incomeCurrencyShortTitle', 
    'changedDate', 
    'payee', 
    'incomeAccountName',
    'income',
], inplace=True, axis=1) 
df['date'] = pd.to_datetime(df['date'])
df['createdDate'] = pd.to_datetime(df['createdDate'])
df['outcome'] = df['outcome'].apply(to_float)

# df.info()
df['month'] = df['date'].apply(lambda i: i.strftime('%B'))
df['createdDate'] = pd.to_datetime(df['createdDate'])
df.loc[df['categoryName'] == 'Кофе', 'categoryName'] = 'Обед'
df.loc[df['categoryName'] == 'Салаты', 'categoryName'] = 'Продукты и домашние покупки'
df['categoryName'] = df['categoryName'].astype('category')

In [ ]:
def some(row):
#     print(type(row['comment']), bool(row['comment']), row['comment'])
    category, comment = row['categoryName'], row['comment']
    if not comment or isinstance(comment, float):
        comment = ''
    comment = comment.lower()

    if category == 'Малышка' and 'няня' in comment:
        return 'nany'
    return category
df['categoryName'] = df.apply(some, axis=1)

In [ ]:
df.head()

In [ ]:
df.head()

In [ ]:
CURR_MONTH = dt.date.today().replace(day=1)
PREV_MONTH = (CURR_MONTH - dt.timedelta(days=10)).replace(day=1)
if PREV_MONTH and 0:
    prev_month_df = df[(df['date'] >= PREV_MONTH) & (df['date'] < CURR_MONTH)]
    print(PREV_MONTH,  ' <= date < ', CURR_MONTH)
    month_df = prev_month_df
else:
    print(CURR_MONTH, ' <= date')
    month_df = df[df['date'] >= CURR_MONTH]

month_sum = month_df['outcome'].sum()
month_sum_without_rent = month_df[month_df['categoryName'] != 'Квартира и коммунальные'].sum()
month_sum, month_sum_without_rent, CURR_MONTH.strftime('%Y-%m-01')


In [ ]:
month_gr = pd.DataFrame(month_df.groupby('categoryName')['outcome'].sum())
month_gr['percents'] = month_gr['outcome'] / month_sum * 100
month_gr.sort_values('outcome', ascending=False, inplace=True)
month_gr.reset_index(inplace=True)
month_gr

In [ ]:
month_name = CURR_MONTH.strftime('%B')
layout = go.Layout(
    title=f'Outcome - {month_name}',
    showlegend=False,   
)

# for i, row in month_gr.iterrows():
#     layout['annotations'].append(
#         dict(
#             x=row['categoryName'], 
#             y=row['outcome'] + 300, 
#             text=f"{row['percents']:0.2f}%",
#             font=dict(family='Arial', size=14),
#             showarrow=False,            
#         )
#     )

# month_gr.iplot(kind='bar',x='categoryName',y='outcome', layout=layout)

In [ ]:
df_pivot = df.pivot_table(index='date', columns='categoryName', values='outcome', aggfunc=sum)
df_pivot.resample('M').sum().iplot(kind='bar', barmode='group')

In [ ]:
df_index = df.set_index('date')
# df_index.resample('M')['outcome'].sum()
df_index[df_index['categoryName'] != 'Заграницу'].resample('M').sum().iplot(kind='bar', barmode='group')

In [ ]:
qgrid.QgridWidget(df=df, show_toolbar=True)

In [ ]:
month_df.sort_values('outcome', ascending=False)[['categoryName', 'comment', 'outcome', 'date']]

In [ ]:
future = {
    "Квартира и коммунальные": 15000,
    "Продукты и домашние покупки": 10000,
    "Кафе и рестораны": 4000,
    "Медицина": 1000,
    "Развлечения, хобби и подарки": 1000,
    "Малышка": 4000, #(Бодики + Коврик + Стульчик)
    "Барбос": 1000,
    "Обед": 2500,
    "Подарок": 800, # (Подарок Насте)    
    "Красота": 2500,
    "Одежда и Обувь": 1500,  #(Футболки Игорю)
    "Транспорт": 2000,
    "Уборка": 1600,
    "Заграницу": 21000,  # AirBNB
    "Налоги": 3000,
    
#     "Спорт и здоровый образ жизни": 11000,  # (Басейн + Спортлайф)    
#     "Долг": 7000,
    "Родители и родственники - поддержка": 3000,    
#     "часы": 7000,
}    
sum(future.values())    
    
    

In [ ]:
df['categoryName'].unique()

In [ ]:
df[df['categoryName']=='Развлечения, хобби и подарки']